# CitiBikes: Data Processing

I process the 2023 data for the NYC citibikes to generate summary monthly numbers per station and thus reduce the size of the csv used to visualize using Tableau.

# Notes

2013 data until now

Dataset 1 -- is bikeshare more popular from 2013 until now
- year
- month
- rideable type
- member_casual
- binned trip duration in mins (calculate in python)
- binned trip distance in meters (calculate in python)
- count(ride_id)
  


In [1]:
# Dependencies
import pandas as pd
from pathlib import Path

In [4]:
# Empty df to appends data in each file
df = pd.DataFrame()

print(f'df length: {len(df)}')

# loop through all monthly files and concatenate them
for i in range(6, 12):
    if i == 0:
        date = '202212'
    else:
        date = f'2023{i:02}'
    print(f'{date} start')
    working_df = pd.read_csv(Path('Resources', f'{date}-citibike-tripdata.csv'), low_memory=False)
    print(f'{date} read')
    working_df = (working_df
        .dropna()
        .groupby([
            'start_station_id',
            'start_station_name',
            'start_lat',
            'start_lng',
            'end_station_id',
            'end_station_name',
            'end_lat',
            'end_lng',
            'member_casual',
            'rideable_type'
        ], as_index=False, sort=False)
        .agg({'ride_id': 'count'})
    )
    print(f'{date} aggregated')
    working_df['month'] = date
    print(f'{date} df length: {len(working_df)}')
    df = pd.concat([df, working_df])
    print(f'{date} concatenation done')
    print(f'total df length: {len(df)}')
del(working_df)

df length: 0
202306 start
202306 read
202306 aggregated
202306 df length: 1813031
202306 concatenation done
total df length: 1813031
202307 start
202307 read
202307 aggregated
202307 df length: 2118642
202307 concatenation done
total df length: 3931673
202308 start
202308 read
202308 aggregated
202308 df length: 2354897
202308 concatenation done
total df length: 6286570
202309 start
202309 read
202309 aggregated
202309 df length: 2114795
202309 concatenation done
total df length: 8401365
202310 start
202310 read
202310 aggregated
202310 df length: 2345817
202310 concatenation done
total df length: 10747182
202311 start
202311 read
202311 aggregated
202311 df length: 1970117
202311 concatenation done
total df length: 12717299


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12717299 entries, 0 to 1970116
Data columns (total 12 columns):
 #   Column              Dtype  
---  ------              -----  
 0   start_station_id    object 
 1   start_station_name  object 
 2   start_lat           float64
 3   start_lng           float64
 4   end_station_id      object 
 5   end_station_name    object 
 6   end_lat             float64
 7   end_lng             float64
 8   member_casual       object 
 9   rideable_type       object 
 10  ride_id             int64  
 11  month               object 
dtypes: float64(4), int64(1), object(7)
memory usage: 1.2+ GB


In [7]:
df.to_csv('202306-202311-citibike-monthly_route_counts.csv', index=False, header=True)


---


In [4]:
df['month'].value_counts(sort=False).to_csv('2023-citibike-month-counts.csv')

In [5]:
# keep only rows with routes that are repeated over multiple months (i.e. it wasn't a one-of in the year)
df_repeated_trips = df.loc[df.drop(columns=['member_casual', 'rideable_type', 'month', 'count']).duplicated()]
print(df_repeated_trips.info())
df_repeated_trips.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4940140 entries, 85 to 1603235
Data columns (total 12 columns):
 #   Column              Dtype  
---  ------              -----  
 0   start_station_name  object 
 1   start_station_id    object 
 2   end_station_name    object 
 3   end_station_id      object 
 4   start_lat           float64
 5   start_lng           float64
 6   end_lat             float64
 7   end_lng             float64
 8   member_casual       object 
 9   rideable_type       object 
 10  month               object 
 11  count               int64  
dtypes: float64(4), int64(1), object(7)
memory usage: 490.0+ MB
None


,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,rideable_type,month,count
85,Central Park S & 6 Ave,6876.04,Central Park S & 6 Ave,6876.04,40.765909,-73.976342,40.765909,-73.976342,casual,electric_bike,January,130
114,7 Ave & Central Park South,6912.01,7 Ave & Central Park South,6912.01,40.766741,-73.979069,40.766741,-73.979069,casual,classic_bike,January,118
229,McGuinness Blvd & Eagle St,5977.01,Vernon Blvd & 50 Ave,6170.02,40.735550,-73.952840,40.742327,-73.954117,member,electric_bike,January,93
276,Central Park S & 6 Ave,6876.04,Central Park S & 6 Ave,6876.04,40.765909,-73.976342,40.765909,-73.976342,member,classic_bike,January,87
299,Grand Army Plaza & Central Park S,6839.10,Grand Army Plaza & Central Park S,6839.10,40.764397,-73.973715,40.764397,-73.973715,casual,electric_bike,January,84


In [6]:
df_repeated_trips.to_csv('2023-citibike-counts_repeated_trips.csv', index=False, header=True)

In [7]:
# only keep stations id out of station info
df_repeated_trips_thin = df_repeated_trips[[
    'start_station_id', 'end_station_id', 'member_casual', 'rideable_type', 'month', 'count'
]]
print(df_repeated_trips_thin.info())
df_repeated_trips_thin.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4940140 entries, 85 to 1603235
Data columns (total 6 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   start_station_id  object
 1   end_station_id    object
 2   member_casual     object
 3   rideable_type     object
 4   month             object
 5   count             int64 
dtypes: int64(1), object(5)
memory usage: 263.8+ MB
None


,start_station_id,end_station_id,member_casual,rideable_type,month,count
85,6876.04,6876.04,casual,electric_bike,January,130
114,6912.01,6912.01,casual,classic_bike,January,118
229,5977.01,6170.02,member,electric_bike,January,93
276,6876.04,6876.04,member,classic_bike,January,87
299,6839.10,6839.10,casual,electric_bike,January,84


In [8]:
df_repeated_trips_thin.to_csv('2023-citibike-counts_repeated_trips_thin.csv', index=False, header=True)

## Further processing

We can get the aggregated desired counts for each column separetely to reduce the size of the csv

In [8]:
# data frame with stations info: id, name, lat and lng
# get only columns with station info
stations_df = (df[[
        'start_station_name',
        'start_station_id',
        'start_lat',
        'start_lng',
        'end_station_name',
        'end_station_id',
        'end_lat',
        'end_lng'
    ]]
)
# rename columns using multuindex, so i can use the stack method later
stations_df.columns = (pd
    .MultiIndex
    .from_tuples(
        [
            ('start', 'name'),
            ('start', 'id'),
            ('start', 'lat'),
            ('start', 'lng'),
            ('end', 'name'),
            ('end', 'id'),
            ('end', 'lat'),
            ('end', 'lng')
        ]
    )
)
print(stations_df.info())
stations_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20506610 entries, 0 to 1603237
Data columns (total 8 columns):
 #   Column         Dtype  
---  ------         -----  
 0   (start, name)  object 
 1   (start, id)    object 
 2   (start, lat)   float64
 3   (start, lng)   float64
 4   (end, name)    object 
 5   (end, id)      object 
 6   (end, lat)     float64
 7   (end, lng)     float64
dtypes: float64(4), object(4)
memory usage: 1.4+ GB
None


start                                 \
                            name       id        lat        lng   
0            St Marks Pl & 2 Ave  5669.10  40.728419 -73.987140   
1                8 Ave & W 33 St  6450.12  40.751551 -73.993934   
2                W 21 St & 6 Ave  6140.05  40.741740 -73.994156   
3  North Moore St & Greenwich St  5470.12  40.720195 -74.010301   
4           Vesey St & Church St  5216.06  40.712220 -74.010472   

                             end                                 
                            name       id        lat        lng  
0            St Marks Pl & 1 Ave  5626.13  40.727791 -73.985649  
1                W 35 St & 8 Ave  6526.01  40.752762 -73.992805  
2                9 Ave & W 22 St  6266.06  40.745497 -74.001971  
3           Vesey St & Church St  5216.06  40.712220 -74.010472  
4  North Moore St & Greenwich St  5470.12  40.720195 -74.010301

In [9]:
# stack df, reset index and drop duplicates
stations_df = (stations_df
    .stack(0)
    .reset_index(drop=True)
    .drop_duplicates(ignore_index=True)
)
print(stations_df.info())
stations_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14334916 entries, 0 to 14334915
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   id      object 
 1   lat     float64
 2   lng     float64
 3   name    object 
dtypes: float64(2), object(2)
memory usage: 437.5+ MB
None


,id,lat,lng,name
0,5626.13,40.727791,-73.985649,St Marks Pl & 1 Ave
1,5669.10,40.728419,-73.987140,St Marks Pl & 2 Ave
2,6526.01,40.752762,-73.992805,W 35 St & 8 Ave
3,6450.12,40.751551,-73.993934,8 Ave & W 33 St
4,6266.06,40.745497,-74.001971,9 Ave & W 22 St


In [11]:
# Output group by month and: (1) start
start_df = (df
    .groupby(['month', 'start_station_id'], as_index=False)
    .agg({'count': 'sum'})
)
print(start_df.info())
start_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23033 entries, 0 to 23032
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   month             23033 non-null  object
 1   start_station_id  23033 non-null  object
 2   count             23033 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 540.0+ KB
None


,month,start_station_id,count
0,April,2733.03,227
1,April,2782.02,323
2,April,2821.05,251
3,April,2832.03,350
4,April,2861.02,148


In [10]:
# Output group by month and: (2) end
end_df = (df
    .groupby(['month', 'end_station_id'], as_index=False)
    .agg({'count': 'sum'})
)
print(end_df.info())
end_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23413 entries, 0 to 23412
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   month           23413 non-null  object
 1   end_station_id  23413 non-null  object
 2   count           23413 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 548.9+ KB
None


,month,end_station_id,count
0,April,190 Morgan,4
1,April,2733.03,222
2,April,2782.02,329
3,April,2821.05,261
4,April,2832.03,355


In [12]:
# Output group by month and: (1) start, (2) end, (3) member_casual
member_df = (df
    .groupby(['month', 'member_casual'], as_index=False)
    .agg({'count': 'sum'})
)
print(member_df.info())
member_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   month          24 non-null     object
 1   member_casual  24 non-null     object
 2   count          24 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 704.0+ bytes
None


,month,member_casual,count
0,April,casual,527556
1,April,member,2319246
2,August,casual,903059
3,August,member,3180113
4,December,casual,337450


In [13]:
# Output group by month and: (4) rideable_type
ride_df = (df
    .groupby(['month', 'rideable_type'], as_index=False)
    .agg({'count': 'sum'})
)
print(ride_df.info())
ride_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   month          31 non-null     object
 1   rideable_type  31 non-null     object
 2   count          31 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 872.0+ bytes
None


,month,rideable_type,count
0,April,classic_bike,2374481
1,April,docked_bike,15183
2,April,electric_bike,457138
3,August,classic_bike,3775211
4,August,electric_bike,307961


In [14]:
stations_df.to_csv('2023-citibike-stations.csv', index=False, header=True)
start_df.to_csv('2023-citibike-start.csv', index=False, header=True)
end_df.to_csv('2023-citibike-end.csv', index=False, header=True)
member_df.to_csv('2023-citibike-member_casual.csv', index=False, header=True)
ride_df.to_csv('2023-citibike-rideable_type.csv', index=False, header=True)